In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in cell average)
def gen_pair_cell_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

In [3]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [4]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h1, h2, o, twolayers):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h1)
        self.relu1 = nn.ReLU()
        self.linear21 = nn.Linear(h1, h2)
        self.relu21 = nn.ReLU()
        self.linear22 = nn.Linear(h2, o)
        self.relu22 = nn.ReLU()
        self.linear23 = nn.Linear(h1, o)
        self.relu23 = nn.ReLU()
        self.twolayers = twolayers
        
    def forward(self, x):
        if twolayers == True:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear21(out)
            out = self.relu21(out)
            out = self.linear22(out)
            out = self.relu22(out)
        else:
            out = self.linear1(x)
            out = self.relu1(out)
            out = self.linear23(out)
            out = self.relu23(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [5]:
def gen_pair_by_xt(delta_x=1/20, delta_y=1/20, xmin=0, ymin=0, xmax=2 * np.pi, ymax=2 * np.pi, length=3, num=10000, code=2):
    x = arange(xmin, xmax, delta_x)
    t = arange(ymin, ymax, delta_y)
    X,T = meshgrid(x, t) # grid of point
    Z = heat_equ_analytical_solu(X, T) # evaluation of the function on the grid
    if code == 0:
        return gen_pair(Z, len(Z[0]), len(Z), length=length, num=num)
    if code == 1:
        return gen_pair_ave(Z, len(Z[0]), len(Z), length=length, num=num)
    if code == 2:
        return gen_pair_cell_ave(Z, len(Z[0]), len(Z), length=length, num=num)
    else:
        return gen_pair(Z, len(Z[0]), len(Z), length=length, num=num)

In [6]:
def trainning_the_model(data_pairs, iteration=2, twolayers=False, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False, loss_file='list_of_loss_11 xx.txt', output_file='list_of_output_11 xx.txt', model_name="model_xx delta x=1 20 delta t=1 20"):
    model = ResNet(7, 6, 6, 1, twolayers=twolayers)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, amsgrad=amsgrad)
    criterion = nn.MSELoss()
    model.train()
    list_of_loss = []
    list_of_output = []
    # train for each data pair
    # by interation=2 times
    for data in data_pairs:
        for itera in range(iteration):
            output = model(torch.FloatTensor(data["input"]))
            loss = criterion(output, torch.FloatTensor([data["solu"]]))
            list_of_loss.append(np.log10(loss.item()))
            list_of_output.append(output)
            model.zero_grad()
            loss.backward()
            optimizer.step()
    # save the list_of_loss
    list_of_loss_file = open(loss_file, 'w+')
    for value in list_of_loss:
        list_of_loss_file.write(str(value)+" ")
    list_of_loss_file.close()
    # save the list_of_output
    list_of_output_file = open(output_file, 'w+')
    for value in list_of_output:
        list_of_output_file.write(str(value)+" ")
    list_of_output_file.close()
    # finally, save the model
    model_1.save_model(model_name)

In [ ]:
pairs_20_20 = gen_pair_by_xt(delta_x=1/20, delta_y=1/20)
pairs_20_40 = gen_pair_by_xt(delta_x=1/20, delta_y=1/40)
pairs_20_80 = gen_pair_by_xt(delta_x=1/20, delta_y=1/80)
pairs_20_160 = gen_pair_by_xt(delta_x=1/20, delta_y=1/160)